In [ ]:
import warnings
#warnings.filterwarnings('ignore')
#warnings.simplefilter('ignore')

#import czifile as zis
from apeer_ometiff_library import io, processing, omexmlClass
#import os
#from matplotlib import pyplot as plt, cm
#from mpl_toolkits.axes_grid1 import make_axes_locatable
#from skimage.viewer import ImageViewer
#import skimage.io
#import matplotlib.colors as colors
import numpy as np
import ipywidgets as widgets
import napari
import imgfileutils as imf

In [ ]:
##### TEST CZI #####

imgdict = {
    1:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Lightsheet_Shoh\Brain_Smartlabel_6x11_Z473_12OL_C0_left_(049).czi',
    2:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\CellDivision_T=15_Z=20_CH=2_DCV.czi',
    3:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\CellDivision_T=10_Z=15_CH=2_DCV_small.czi',
    4:r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\CellDivision_T=10_Z=15_CH=2_DCV_small.ome.tiff',
    5: r'C:\Users\m1srh\Documents\Testdata_Zeiss\Castor\Z-Stack_DCV\CellDivision_T=10_Z=15_CH=2_DCV_small_Fiji.ome.tiff'
}

In [ ]:
filename = imgdict[4]
image_name = os.path.basename(filename)

if filename.lower().endswith('.ome.tiff') or filename.lower().endswith('.ome.tif'):
    
    # Return value is an array of order (T, Z, C, X, Y)
    (array, omexml) = io.read_ometiff(filename)
    metadata = imf.get_metadata(filename, series=0)
    
if filename.lower().endswith('.czi'):
    
    # get only the metadata
    metadata = imf.get_metadata(filename)
    
    print(metadata['Shape'])
    print(metadata['Axes'])
    
    # get the array and the metadata
    array, metadata = imf.get_array_czi(filename, replacezero=False)

    print(array.shape)

In [ ]:
# show some metadata

print(metadata['Shape'])
print(metadata['Axes'])
print(metadata['DimOrder BF'])
print(metadata['DimOrder BF Array'])
print(array.shape)
print(metadata['Extension'])

In [ ]:
show_ipyviewer = False

if show_ipyviewer:

    #out = widgets.interactive_output(get_TZC, { 't': t, 'z': z, 'c': c, 'r':r})
    if metadata['Extension'] == 'ome.tiff':
        ui, out = imf.create_ipyviewer_ome_tiff(array, metadata)
    if metadata['Extension'] == 'czi':
        ui, out = imf.create_ipyviewer_czi(array, metadata)

    display(ui, out)

In [ ]:
import time

# switch to qt5 backend for napari viewer and wait a few seconds
%gui qt5
time.sleep(5)

In [ ]:
napari_auto = True

# try to configre napari automatiaclly based on metadata
if napari_auto:
    imf.show_napari(array, metadata)

# configure napari viewer manually - check array diemsnion and dimensio order carefully 
if not napari_auto:
    
    # get the scalefactors
    scalefactors = imf.get_scalefactor(metadata)
    print(scalefactors)
    
    viewer = napari.Viewer()
    # add every channel as a single layer
    for ch in range(metadata['SizeC']):
        chname = metadata['Channels'][ch]
        viewer.add_image(array[ch, :, :, :, :], name=chname, scale=scalefactors['zx'])